In [1]:
import io
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from pathlib import Path
from PIL import Image 

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

2024-07-12 14:00:01.007041: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 14:00:01.009389: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 14:00:01.017515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 14:00:01.034038: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 14:00:01.034064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 14:00:01.044044: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
path = Path("analyze.ipynb")
project_path = str(path.parent.absolute().parent)

In [3]:
def read_csv(path: str) -> pd.DataFrame:
    return pd.read_csv(os.path.join(project_path, path))

def read_image(image_name: str) -> Image:
    with h5py.File(os.path.join(project_path, r"data\raw", "train-image.hdf5"), "r") as f:
        im = Image.open(io.BytesIO(f[image_name][()]))
    return im

def load_hdf5(path: str):
    with h5py.File(os.path.join(project_path, path), "r") as f:
        file = f
    return file

In [4]:
train_hdf5 = load_hdf5("data/raw/train-image.hdf5")
train_hdf5

<Closed HDF5 file>

In [5]:
with h5py.File(os.path.join(project_path, r"data/raw/train-image.hdf5"), "r") as f:
    keys = list(f.keys())[:10]
    labels = [1] * 10
    data = []
    for i in keys:
        im = Image.open(io.BytesIO(f[i][()]))
        im = im.resize((128, 128))
        pix = np.array(im)
        data.append(pix)
        print(pix.shape)
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))

dataset


(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)


<_TensorSliceDataset element_spec=(TensorSpec(shape=(128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [28]:
def generator():
    with h5py.File(os.path.join(project_path, r"data/raw/train-image.hdf5"), "r") as f:
        keys = list(f.keys())[:10]
        labels = [0] * 10
        for i in keys:
            im = Image.open(io.BytesIO(f[i][()]))
            im = im.resize((64, 64))
            pix = np.array(im)
            
            yield pix, np.array(labels[0])

In [29]:
next(iter(generator()))

(array([[[182, 148, 136],
         [183, 149, 137],
         [186, 149, 137],
         ...,
         [189, 148, 141],
         [191, 153, 144],
         [188, 151, 141]],
 
        [[192, 156, 144],
         [191, 155, 143],
         [189, 153, 141],
         ...,
         [185, 145, 137],
         [183, 145, 136],
         [183, 145, 136]],
 
        [[190, 153, 139],
         [188, 151, 138],
         [186, 148, 135],
         ...,
         [179, 141, 132],
         [177, 139, 130],
         [179, 140, 131]],
 
        ...,
 
        [[182, 145, 140],
         [180, 142, 135],
         [178, 141, 131],
         ...,
         [183, 141, 129],
         [182, 140, 128],
         [179, 137, 125]],
 
        [[184, 145, 139],
         [177, 137, 129],
         [176, 137, 126],
         ...,
         [182, 139, 126],
         [181, 138, 126],
         [176, 133, 121]],
 
        [[186, 145, 139],
         [176, 136, 128],
         [176, 137, 124],
         ...,
         [177, 133, 120],
  

In [31]:
ds = tf.data.Dataset.from_generator(generator,
                                output_types=(tf.uint8, tf.int32)
                               )

ds = ds.batch(2)

for i in ds.take(1):
    print(i[0].shape.as_list())
    print(i[1].shape.as_list())

[2, 64, 64, 3]
[2]


2024-07-12 14:32:11.464951: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [32]:
model = tf.keras.applications.ResNet50(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(64, 64, 3),
    pooling='avg',
    classes=2,
    classifier_activation='softmax'
)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [34]:
tf.keras.config.disable_traceback_filtering()

In [35]:
model.fit(
    ds, 
    epochs=1
)

ValueError: as_list() is not defined on an unknown TensorShape.